# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("./city_weather.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,New Norfolk,-42.7826,147.0587,51.71,80,100,3.00,AU,1661885990
1,1,Constitución,-35.3333,-72.4167,57.47,71,68,7.90,CL,1661885991
2,2,Cherskiy,68.7500,161.3000,34.66,89,23,12.48,RU,1661885991
3,3,Ushuaia,-54.8000,-68.3000,49.66,57,75,5.75,AR,1661885992
4,4,Baní,18.2833,-70.3333,90.84,55,99,12.30,DO,1661885992


In [3]:
weather_df.count()

Unnamed: 0    591
City          591
Lat           591
Lng           591
Max Temp      591
Humidity      591
Cloudiness    591
Wind Speed    591
Country       585
Date          591
dtype: int64

In [4]:
del weather_df['Unnamed: 0']

In [5]:
# Drop missing values
weather_df_no_missing = weather_df.dropna()
weather_df_no_missing.reset_index(drop=False, inplace=False)
weather_df_no_missing.count()

City          585
Lat           585
Lng           585
Max Temp      585
Humidity      585
Cloudiness    585
Wind Speed    585
Country       585
Date          585
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
figure_layout = {
    'width': '1920px',
    'height': '1080px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heatmap_humidity = gmaps.figure(map_type="HYBRID", layout=figure_layout)
max_int = weather_df["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations=weather_df_no_missing[['Lat', 'Lng']], weights=weather_df_no_missing['Humidity'].astype(float), dissipating=False,
    max_intensity=max_int, point_radius=1)
heatmap_humidity.add_layer(heat_layer)
heatmap_humidity

<Figure size 432x288 with 0 Axes>

In [11]:
heatmap_humidity

Figure(layout=FigureLayout(height='1080px', margin='0 auto 0 auto', padding='1px', width='1920px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Weather conditions:
# Max Temp: between 65 and 70 degrees
# Wind Speed: 10 mph
# Cloudiness: 40-50%

perfect_weather = weather_df_no_missing.loc[(weather_df_no_missing["Max Temp"] >=60)&
    (weather_df_no_missing["Max Temp"]<=70)&
    (weather_df_no_missing["Wind Speed"]<=10)&
    (weather_df_no_missing["Cloudiness"]<=50)&
    (weather_df_no_missing["Cloudiness"]>=20), :]
perfect_weather.reset_index(drop=False, inplace=False)
perfect_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Pisco,-13.7000,-76.2167,64.45,68,40,9.22,PE,1661885702
264,Ampanihy,-24.7000,44.7500,65.62,81,47,8.79,MG,1661886096
265,Shebunino,46.4339,141.8603,62.74,91,24,4.23,RU,1661886097
355,Rohru,31.2167,77.7500,63.81,95,22,1.63,IN,1661886129
461,Manakara,-22.1333,48.0167,68.38,96,31,5.70,MG,1661886168


In [13]:
perfect_weather.count()

City          9
Lat           9
Lng           9
Max Temp      9
Humidity      9
Cloudiness    9
Wind Speed    9
Country       9
Date          9
dtype: int64

In [14]:
perfect_weather.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = perfect_weather.copy()
hotel_df["Hotel Name"]=""
hotel_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Hotel Name'],
      dtype='object')

In [17]:
for i, r in hotel_df.iterrows():
    h_lat = r["Lat"]
    h_lng = r["Lng"]
    hotel_coord = f"{h_lat},{h_lng}"
    params = {
        "location": hotel_coord,
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    response=requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    try:
        hotel_df.loc[i, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print(f'No hotel name available in {hotel_df["City"][i]}!')

No hotel name available in Shebunino!
No hotel name available in Poya!


In [18]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Pisco,-13.7000,-76.2167,64.45,68,40,9.22,PE,1661885702,Hotel San Isidro Oficial
264,Ampanihy,-24.7000,44.7500,65.62,81,47,8.79,MG,1661886096,HOTEL SHARRON AMPANIHY OUEST
265,Shebunino,46.4339,141.8603,62.74,91,24,4.23,RU,1661886097,
355,Rohru,31.2167,77.7500,63.81,95,22,1.63,IN,1661886129,The Pavilion Rohru
461,Manakara,-22.1333,48.0167,68.38,96,31,5.70,MG,1661886168,Hotel H1 Manakara


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations)
heatmap_humidity.add_layer(markers)


<Figure size 432x288 with 0 Axes>

In [25]:
heatmap_humidity

Figure(layout=FigureLayout(height='1080px', margin='0 auto 0 auto', padding='1px', width='1920px'))